# Synthetix Perpetual Futures

In [1]:
from variables import *
import numpy as np
import matplotlib.pyplot as plt

## Parameters

### Market and Position Parameters

#### Position Size
* Measured in units of the base asset (e.g. position "worth" 10 $BTC will have `position_size = 10`)
* Long positions are positive, short positions are negative

In [2]:
position_size = (initial_margin * initial_leverage) / initial_base_asset_spot_price

#### Notional Value
* The (signed) value of the base currency units on a position
* long positions are positive, short positions are negative

Current Notional Value

In [3]:
notional_value = position_size * base_asset_spot_price

Initial Notional Value

In [4]:

initial_notional_value = initial_margin * initial_leverage

#### Profit and Loss (PnL)
* The profit in a position is the change in its notional value since entry

In [5]:
profit_and_loss = notional_value - initial_notional_value

#### Market Size
* Total size of all outstanding positions in the market

In [6]:
market_size = sum(set_of_all_positions_in_the_market) if sum(set_of_all_positions_in_the_market) == set_of_all_long_positions_in_the_market + set_of_all_short_positions_in_the_market else Exception("Market size calculation error")

#### Market skew
* Ratio of long to short positions in the market (relative to position_size)
* Positive skew means more longs than shorts
* Negative skew means more shorts than longs

In [7]:
def get_market_skew():
    skew = 0
    for position in set_of_all_positions_in_the_market:
        skew += position.size
    return skew

market_skew = get_market_skew()

### Leverage and Margins Parameters

#### Leverage
* The ratio of notional value to margin
* λ = v / m

Current Leverage

In [8]:
leverage = notional_value / remaining_margin

Initial Leverage

In [9]:
initial_leverage = initial_notional_value / initial_margin

#### Margin

Initial Margin

In [10]:
initial_margin = position_size * initial_base_asset_spot_price / initial_leverage

Remaining Margin
* if remaining_margin < 0, position is liquidated


In [11]:
remaining_margin = max(initial_margin + profit_and_loss + accrued_position_funding, 0)

Position Debt
* Margin "owed" to the "system" (i.e. margin that must be paid back to close the position)
  * When a position is closed, the funds in its margin are settled. After profit and funding are computed, the remaining margin will be minted into the account that created the position, while any losses out of the initial margin, will be minted into the fee pool."

In [12]:
position_debt = abs(initial_notional_value) - initial_margin

### Exchange Fees
* Fees are imposed when a position is (1) opened, (2) increased

#### Maker
* A maker is a trader who is reducing skew in the market. 
  * If the market is long skew, a maker is a trader who is opening a short position. If the market is short skew, a maker is a trader who is opening a long position.

#### Taker
* A taker is a trader who is increasing skew in the market. 
  * If the market is long skew, a taker is a trader who is opening a long position. If the market is short skew, a taker is a trader who is opening a short position.


#### Fees
* fees will be charged out of the user's remaining margin
  * since fees are charged out of the remaining margin, they will affect the user's leverage

### Skew Funding Rate

#### Proportional Skew
* The skew normalized by the skew scaling constant.

In [13]:
proportional_skew = market_skew / skew_scaling_denominator_constant

### Aggregate Debt Calculation

#### Total Debt
* The overall market debt is the sum of the remaining margin in all positions

Aggregate Position Entry Debt Correction

In [14]:
aggregate_position_entry_debt_correction = (
    market_skew * (base_asset_spot_price + unrecorded_cumulative_funding)
) + np.sum([
    (
        position.initial_margin 
        - position.initial_notional_value 
        - (position.position_size * position.unrecorded_cumulative_funding_at_last_modified_index)
    ) for position in set_of_all_positions_in_the_market
])

Market Debt

In [15]:
market_debt = (
    max(
        market_skew * (base_asset_spot_price + unrecorded_cumulative_funding) 
        + aggregate_position_entry_debt_correction, 
        0
    )
)

### Liquidations and Keepers

#### Liquidation margin
* Liquidation margin below which a position will be liquidatable

In [16]:
liquidation_margin = (
    max(
        position_size * base_asset_spot_price * liquidation_fee_ratio,
        minimal_keeper_incentive
    )
    + (position_size * base_asset_spot_price * liquidation_buffer_ratio)
)

Minimal keeper incentive
* The minimum incentive for a keeper to liquidate a position (in futures v1 this value was set to 20 $sUSD)

Liquidation buffer ratio
* Ratio of notional position size that will be used as buffer to reduce chance of negative margin on large price updates

Liquidation fee ratio
* Ratio of notional position size that will be paid to incentive liquidation keepers

Approxime liquidation price
* Approximate liquidation price as estimated using liquidation margin calculated for current price
* Will be above actual liquidation price for longs, and below for shorts

In [17]:
approximate_liquidation_price = (
    initial_base_asset_spot_price
    - ((initial_margin - liquidation_margin) / position_size)
    - (unrecorded_cumulative_funding - unrecorded_cumulative_funding_at_last_modified_index)
)